# Data: Joining Datasets

*Purpose*: Often our data are scattered across multiple sets. In this case, we need to be able to *join* data.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

# Danger! Naive "binding" of data


The simplest means we have to combine two datasets is to *bind* them together. The verb `gr.tf_bind_rows()` binds together two datasets vertically (adds rows to rows), while `gr.tf_bind_cols()` binds two datasets horizontally (adds new columns to the dataset). These are very simple ways to combine datasets; for example:


In [ ]:
## NOTE: No need to edit
(
    gr.df_make(numbers=[1,2,3,4])
    >> gr.tf_bind_cols(
        gr.df_make(letters=["A", "B", "C", "D"])
    )
)

Binding is appropriate when we have *strong knowledge* of how our data are structured; we blindly smash together rows or columns to make a new dataframe, so we'd better be sure those rows/columns are in the right order! Otherwise, we may get surprising (and wrong) results....


### __q1__ What went wrong with this bind?

Run the following code and answer the questions under *observations* below.

*Hint*: If you're not a Beatles fan, it may be helpful to consult the relevant [personnel](https://en.wikipedia.org/wiki/The_Beatles#Personnel) page on Wikipedia.


In [ ]:
## NOTE: No need to edit; run and inspect
# Setup
df_beatles = gr.df_make(
    band=["Beatles"] * 4,
    name=["John", "Paul", "George", "Ringo"],
)

df_beatles_instruments = gr.df_make(
    surname=["McCartney", "Harrison", "Starr", "Lennon"],
    instrument=["bass", "guitar", "drums", "guitar"]
)

# Attempt to combine the datasets... to disastrous results!
(
    df_beatles
    >> gr.tf_bind_cols(df_beatles_instruments)
)

*Observations*

- What went wrong in binding `df_beatles` and `df_beatles_instruments`?
  - (Your response here)


# A safer way: "Joining" datasets

A safer way to combine two datasets is to *not assume they are ordered correctly*. Instead, we can use *common* information to *join* two datasets. In order to do a join, we must have a set of "keys" by which to combine data from the two datasets. For instance, if we had a DataFrame with both `name` and `surname`, we could join to `df_beatles1` by the `name` column.


In [ ]:
## NOTE: No need to edit
df_beatles_surnames = gr.df_make(
    name=["John", "Paul", "George", "Ringo"],
    surname=["Lennon", "McCartney", "Harrison", "Starr"],
)

df_beatles_names = (
    df_beatles
    >> gr.tf_left_join(df_beatles_surnames, by="name")
)
df_beatles_names

Note that this correctly associates names with surnames.


### __q2__ Do a join

Use `gr.tf_left_join()` to associate each `instrument` with the correct band member.


In [ ]:
## TASK: Join df_beatles2 correctly to add the `instrument` column
df_beatles_full = (
    df_beatles_names

)

## NOTE: Use this to check your work
assert \
    "instrument" in df_beatles_full.columns, \
    "df_beatles_full does not have an `instrument` column"

assert \
    df_beatles_full[df_beatles_full.name == "Ringo"].instrument.values[0] == "drums", \
    "Ringo Starr played drums!"

df_beatles_full


There's a **very important lesson** here: In general, don't trust `gr.tf_bind_cols`. It's easy in the example above to tell there's a problem because the data are *small*; when working with larger datasets, the software will happily give you the wrong answer if you give it the wrong instructions. Whenever possible, use some form of join to combine datasets.


# Types of joins

There are several types of joins:

| Name | Type | Description |
|------|------|-------------|
| `gr.tf_left_join()` | Mutating join | Preserves rows in the left DataFrame being joined |
| `gr.tf_right_join()` | Mutating join | Preserves rows in the right DataFrame being joined |
| `gr.tf_inner_join()` | Mutating join | Preserves rows common to both DataFrames being joined |
| `gr.tf_outer_join()` | Mutating join | Preserves all rows in both DataFrames being joined |
| `gr.tf_semi_join()` | Filtering join | Returns all rows in left DataFrame that have a match in the right DataFrame |
| `gr.tf_anti_join()` | Filtering join | Returns all rows in left DataFrame that have *no* match in the right DataFrame |

We'll discuss (and use!) each of these below.


## Mutating joins

A *mutating join* is a join that also performs a mutation---it adds columns to the DataFrame. Like we saw above, we can use a mutating join to add information to a datset. However, there are four different types of mutating


In [ ]:
## NOTE: No need to edit
df_beatles_names = gr.df_make(
    name=["John", "Paul", "George", "Ringo", "George"],
    surname=["Lennon", "McCartney", "Harrison", "Starr", "Martin"],
)

df_beatles_roles = gr.df_make(
    surname=["Lennon", "McCartney", "Harrison", "Starr", "Epstein"],
    role=["Bandmate", "Bandmate", "Bandmate", "Bandmate", "Manager"],
)


You'll investigate how the various join types function in the next task:


### __q3__ Test the joins

Uncomment one line at a time and run the code below. Answer the questions under *observations* below.


In [ ]:
## TASK: Uncomment one line at a time and run; document your findings
(
    df_beatles_names
    >> gr.tf_left_join(df_beatles_roles, by="surname")
    # >> gr.tf_right_join(df_beatles_roles, by="surname")
    # >> gr.tf_inner_join(df_beatles_roles, by="surname")
    # >> gr.tf_outer_join(df_beatles_roles, by="surname")
)

*Observations*

- Which rows does `tf_left_join()` preserve?
  - (Your response here)
- Which rows does `tf_right_join()` preserve?
  - (Your response here)
- Which rows does `tf_inner_join()` preserve?
  - (Your response here)
- Which rows does `tf_outer_join()` preserve?
  - (Your response here)


### Visual Aid: Types of Joins

The following visual may help you make sense of the four mutating joins; it depicts the four verbs as Venn diagrams for the left (`L`) and right (`R`) DataFrames in the join.

![Venn diagrams for four types of joins: left, right, inner, outer](images/joins.jpg)

You may also find [this image](https://github.com/amartinson193/SQL_Checkered_Flag_Join_Diagrams/blob/main/checkered_flag_diagram_pg1.png) helpful for visualizing the join types.


## Danger! Non-unique keys

Note that when we do any sort of join, we need *unique* keys. We'll run into trouble if the provided keys do not uniquely identify each row. For example, Harrison and Martin share a given name:


In [ ]:
## NOTE: No need to edit
df_beatles_names


Look at what happens when we join on first `name` only:


In [ ]:
## NOTE: No need to edit; this gives incorrect results due to non-unique keys
(
    df_beatles_names
    >> gr.tf_full_join(
        gr.df_make(
            name=["Paul", "George", "Ringo", "John"],
            instrument=["bass", "guitar", "drums", "guitar"]
        ),
        by="name"
    )
)

George Martin didn't play the guitar in the Beatles! He was their producer.

If a single key is not unique, we can use *multiple keys* for the join:


In [ ]:
## NOTE: No need to edit; using multiple keys corrects the issue
(
    df_beatles_names
    >> gr.tf_full_join(
        gr.df_make(
            name=["Paul", "George", "Ringo", "John"],
            surname=["McCartney", "Harrison", "Starr", "Lennon"],
            instrument=["bass", "guitar", "drums", "guitar"]
        ),
        by=["name", "surname"],
    )
)

## Filtering joins

Mutating joins add new columns, but *filtering joins* simply filter the DataFrame. A filter join is particularly helpful when a filter is difficult to express in `gr.tf_filter()`, but easy to express as a set of keys (perhaps with multiple key columns).

As a first example, we can filter on all of the `"George"`'s with a `gr.tf_semi_join()`.


In [ ]:
## NOTE: No need to edit
(
    df_beatles_names
    >> gr.tf_semi_join(
        gr.df_make(name=["George"]),
        by="name",
    )
)

As we saw before, a single key will often not be enough to uniquely identify a row. For instance, the following will filter down to a numer of non-Beatle players:


In [ ]:
## NOTE: No need to edit
(
    gr.df_make(
        surname=["Clapton", "Harrison", "Shankar", "Wooten", "McCartney"],
        instrument=["guitar", "guitar", "sitar", "bass", "bass"],
    )
    >> gr.tf_semi_join(
        df_beatles_instruments,
        by="instrument"
    )
)

### __q4__ Semi-join with multiple keys

Construct the `by` argument for `gr.tf_semi_join()` below to filter to *only* persons who were in the Beatles.


In [ ]:
## TASK: Construct the `by` argument below to filter to *only* persons who were in the Beatles
(
    gr.df_make(
        surname=["Clapton", "Harrison", "Shankar", "Wooten", "McCartney"],
        instrument=["guitar", "guitar", "sitar", "bass", "bass"],
    )
    >> gr.tf_semi_join(
        df_beatles_instruments,
        ## TODO: Construct the proper `by` argument

    )
)

# Going Further: Airports dataset

We'll use the `nycflights13` package to demonstrate joins in a more realistic situation. This is a dataset of flights involving the New York City area during 2013.


In [ ]:
from nycflights13 import flights as df_flights
df_flights

### __q5__ Make a "grid" of filter criteria

Use `gr.df_grid()` to make a DataFrame with the rows.

| `month` | `dest` |
|---------|--------|
|    8    |  "SJC" |
|    8    |  "SFO" |
|    8    |  "OAK" |
|    9    |  "SJC" |
|    9    |  "SFO" |
|    9    |  "OAK" |

*Note*: We'll use this grid soon in a filtering join.


In [ ]:
## TASK: Use gr.df_grid() to make the DataFrame described above
df_criteria = None


## NOTE: Use this to check your work
assert \
    "month" in df_criteria.columns, \
    "df_criteria does not have a 'month' column"

assert \
    "dest" in df_criteria.columns, \
    "df_criteria does not have a 'dest' column"

assert \
    df_criteria.shape[0] == 6, \
    "df_criteria has the wrong number of columns"

df_criteria

There are *many* carriers in this dataset; let's figure out which carriers are associated with flights in August (`month==8`) and September (`month==9`) to the San Francisco Bay Area (SJC, SFO, OAK).


In [ ]:
(
    df_flights
    >> gr.tf_count(DF.carrier)
)

### __q6__ Count the Bay area carriers

Perform a semi-join to filter `df_flights` using all the columns of `df_criteria`. Answer the questions under *observations* below.


In [ ]:
## TASK: Semi-join with df_criteria
(
    df_flights
    ## TODO: Do your semi-join here

    >> gr.tf_count(DF.carrier)
)

*Observations*

- How many carriers provided flights to the SF Bay Area in the months considered?
  - (Your response here)
- How does this number of carriers compare with the total number of carriers?
  - (Your response here)


Unless you work in the airline industry, it's probably difficult to make sense of these carrier codes. Thankfully the `nycflights13` package comes with a dataset that helps disambiguate these codes:


In [ ]:
from nycflights13 import airlines as df_airlines
df_airlines

### __q7__ Make the data more interpretable

Use the appropriate kind of join to add the `name` column to the results below. Answer the questions under *observations* below.


In [ ]:
## TASK: Add the `name` column from `df_airlines`
(
    df_flights
    >> gr.tf_semi_join(df_criteria, by=["month", "dest"])
    >> gr.tf_count(DF.carrier)

)

*Observations*

- Which carrier had the most flights in subset considered?
  - (Your response here)
